# **STUDENT AI** - DATA CLEANING

## Objectives

Inspect the dataset and solve any issues that might arise from wrong data types, or missing / wrong values

## Inputs

Continues to assess dataset loaded in previous notebook.

## Outputs

Saves the cleaned dataset back to inputs/dataset folder for futher use


---

# Import required libraries

In [1]:
import os
import pandas as pd
from pandas_profiling import ProfileReport

print('All Libraries Loaded')

PydanticImportError: `BaseSettings` has been moved to the `pydantic-settings` package. See https://docs.pydantic.dev/2.6/migration/#basesettings-has-moved-to-pydantic-settings for more details.

For further information visit https://errors.pydantic.dev/2.6/u/import-error

# Change working directory

### Set the working directory to notebook parent folder
If the output does not match, click **'clear all outputs'** and then **'restart'** the notebook. 
Then run cells from top to bottom.

In [ ]:
current_dir = os.getcwd()
os.chdir(os.path.dirname(current_dir))
current_dir = os.getcwd()
print('If correct, Active Directory should read: /workspace/student-AI')
print(f"Active Directory: {current_dir}")

### Load saved dataset

In [ ]:
df = pd.read_csv(f"inputs/dataset/Expanded_data_with_more_features.csv")
df.head()

### Drop Unnamed column that pandas created on import

In [ ]:
df.drop(columns=['Unnamed: 0'], inplace=True)
df.head()

### Summarize Dataset Again

In [ ]:

df.info()

At first glance, 30000+ data rows is a very robust dataset to be able to train an ML. <br>
There seems to be no reason why the NrSiblings should be a float data type as you cannot have 0.25 of a brother/sister
I will convert nrSiblings to a string type as it needs to be treated as a categorcial valriable and not a numerical one. - however this throws an error as the dataset seems to have NaN or Inf data, so first we deal with missing data

### Check for missing values

In [ ]:
missing_values = df.isnull().sum()
percentage_missing = (missing_values / len(df)) * 100
percentage_missing = percentage_missing.round(1)
missing_data = pd.DataFrame({'Missing_Values': missing_values, 'Percentage_Missing': percentage_missing})
missing_data['Percentage_Missing'] = missing_data['Percentage_Missing'].astype(str) + '%'
missing_data

### Assess how many rows would need to be dropped...

In [ ]:
dropped_data = df.dropna()
dropped_data.info()

## Initial Assumptions

* Dropping null values leaves us with 19243 data rows ... I hypothesize that while this seems like enough data to still achieve the business requirement, dropping this many values will likely induce an imbalance to the data and bias the dataset. If confirmed, then imputing logical values to fill the gaps is more advisable.
* The Pandas report shows **Alerts** under the nrSiblings variable as it assumes that a value of zero (0) could be a problem. In this case, it can be ignored, as a value of zero (0) is valid and indicates an only child.


## Assessing imbalance on 'dropped' dataset

In [ ]:
pandas_report = ProfileReport(df=dropped_data, minimal=True)
pandas_report.to_notebook_iframe()

## Pandas Report on 'dropped' dataset

Clicking on each variable to view the specific report fo each variable:

| Variable |  Obvious Insights  | 
|---|---|
| Gender  |  shows equal disribution of 50.8% to 49.2% and no missing values |
|EthnicGroup|report is not much value as it gives the word 'group' most emphasis - **string 'group ' needs to be removed from data**|
|ParentEduc|again here the word 'some' is not accurate. this categorical variable should be changed to a linear numerical one indicating 0 none to n highest level of education, with 'some as intermediary values|
|LunchType|this feature is imbalanced with 64.8% standard and 35.2% free/reduced. Will require some feature engineering|
|TestPrep|this feature is imbalanced with 65.4% completed and 34.6% none. Will require some feature engineering|
|ParentMaritalStatus|this feature is imbalanced with the majority rows (57.2%) labelled married|
|practiceSport|this feature is imbalanced with the majority rows (50.5%) labelled sometimes will need to be engineered or dropped|
|IsFirstChild|this feature is imbalanced with 64.5% yes and 35.6% no. Will require some feature engineering|
|NrSiblings|this numerical feature looks skewed and has a few outliers that might be able to be removed - further analysis required|
|TransportMeans|this feature is slightly imbalanced 58.6% private vs 41.4% school_bus|
|WklyStudyHours|this is a categorical feature with options <5, 5-10 ,and >10 hours ... distribution is reasonable balanced 39.3%/32.4%/28.3%|
|MathScore|at first glance looks like normal distribution, but obvious missing values that were dropped|
|ReadingScore|at first glance looks like normal distribution, but more obvious missing values that were dropped|
|WritingScore|at first glance looks like normal distribution, but some missing values that were dropped|


### dropping the missing value data rows has indeed induced an imbalance to the dataset. I will therefore assess the viability of imputing data to fill the gaps...

## Imputing Missing Values
let's review the missing_data form the full dataset

In [ ]:
missing_data

The highest missing data rate is TransportMeans at 10.2%. Thinking logically about how this feature might affect overall school performance of a student, it seems that this is an indicator of economic status of the family.
Another indicator of economic status of the family is LunchType, indicating whether the family has the means available to pay for own school lunches. This feature has no missing values.

I will therefore **drop the TransportMeans** feature as it has many missing values and a single indicator of Family economic status should be sufficient in assessing its effect on school performance.

In [ ]:
df.drop(columns=['TransportMeans'], inplace=True)

### Imputing Categorical Variables

Categorical variables with missing values are :
* 'EthnicGroup'
* 'TestPrep'
* 'ParentEduc'
* 'ParentMaritalStatus'
* 'IsFirstChild'
* 'PracticeSport'
* 'WklyStudyHours'
* ('LunchType' and 'Gender' have no missing values and do not need to be adjusted)

For these categorical variables I will insert the most common value from the dataset (mode) as that will be closest to the actual value probabalistically.

In [ ]:
for column in ['EthnicGroup', 'TestPrep', 'ParentEduc', 'ParentMaritalStatus', 'IsFirstChild', 'PracticeSport', 'WklyStudyHours']:
    mode_value = df[column].mode()[0]
    df[column].fillna(mode_value, inplace=True)

### Imputing Numerical Variables

The only numerical variable from the feature set (not counting the scores which have no missing values) is NrSiblings.
Once the missing values have been imputed, I can also change the data type to a more sensical integer rather than float.
The imputed values will be based on the **median** instead of the **mean** as this is less sensitive to outliers since the variable does contain some 'extreme' values of 6 or more siblings.

In [ ]:
median_value = df['NrSiblings'].median()
df['NrSiblings'] = df['NrSiblings'].fillna(median_value).astype(int)

### Quick check for remaining missing values and check datatype change and possible duplicate values:

In [ ]:
df.isnull().sum()

In [ ]:
df['NrSiblings'].dtype

### Convert nrSiblings Variable to categorical by changing dtype to  string

In [ ]:
df['NrSiblings'] = df['NrSiblings'].astype(str)
df['NrSiblings'].dtype

### The data is now clean and logical values have been added, in the next book I will conduct an EDA to go into detail about the feature set and data distribution / balance. 

## Save file to repository for follow on notebooks

In [ ]:
file_path = 'outputs/dataset/Expanded_data_with_more_features_clean.csv'

# Remove previous file if it exists
if os.path.exists(file_path):
    os.remove(file_path)

# Create the directory if it doesn't exist
os.makedirs(name='outputs/dataset', exist_ok=True)

# Save cleaned DataFrame to the file path
df.to_csv(file_path, index=False)